In [1]:
import pandas as pd
import glob
import datetime
import numpy as np

In [2]:
folder_file_pattern=glob.glob("/home/jian/Projects/Big_Lots/Sales_All/Sales_Data/*.txt")

In [3]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|",dtype=str)
depart_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding = "ISO-8859-1")

In [4]:
depart_id=depart_name[depart_name['department_desc']=="LAWN & GARDEN"]['department_id'].unique()[0]
lawn_garden_tax=taxonomy[taxonomy['department_id']==depart_id]
lawn_garden_tax=lawn_garden_tax[['department_id','class_code_id']]
lawn_garden_tax=lawn_garden_tax.drop_duplicates().reset_index()
del lawn_garden_tax['index']
lawn_garden_tax=pd.merge(lawn_garden_tax,class_name,on="class_code_id",how="left")
lawn_garden_id=lawn_garden_tax['class_code_id'].unique().tolist()

In [9]:
lawn_garden_id_df=pd.DataFrame(columns={"id"},index=range(len(lawn_garden_id)))
lawn_garden_id_df['id']=lawn_garden_id
lawn_garden_id_df.to_csv("/home/jian/lawn_code.csv")

In [5]:
depart_sales_output=pd.DataFrame()
for file in folder_file_pattern:
    df=pd.read_csv(file,sep="|",dtype=str)
    if "class_code_id" in df.columns:

        df=df[df['class_code_id'].isin(lawn_garden_id)]
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].fillna(0.0)
        df=df[['location_id','week_end_dt','class_code_id','class_gross_sales_amt']].drop_duplicates()
        df=df.groupby(['location_id','week_end_dt'])['class_gross_sales_amt'].sum().to_frame()
        df.reset_index(inplace=True)
        try:
            df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
            
        except:
            print(file,"Date Error")
            
            
    else:
        print (file,"Error: class_code_id not in columns")
        
    depart_sales_output=depart_sales_output.append(df)

In [6]:
depart_sales_output.columns=['location_id','week_end_dt','Lawn_and_Garden_Sales']
depart_sales_output_wide=depart_sales_output.pivot(index='location_id', columns='week_end_dt', values='Lawn_and_Garden_Sales')
depart_sales_output_wide=depart_sales_output_wide.fillna(0)
depart_sales_output_wide.reset_index(inplace=True)
depart_sales_output_wide['location_id']=depart_sales_output_wide['location_id'].astype(int)
depart_sales_output_wide=depart_sales_output_wide.sort_values('location_id')
depart_sales_output_wide=depart_sales_output_wide[~depart_sales_output_wide['location_id'].isin([145,6990])]

In [7]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Sales_All/Law_and_Garden_Q1.xlsx")

In [8]:
all_sales_by_store=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_2018-04-21/wide_sales_date2018-04-21.xlsx",sheetname="sales")
all_sales_by_store.to_excel(writer,"all sales",index=False)

In [9]:
all_sales_by_store.columns=["location_id"]+["All_Sales_"+name for name in all_sales_by_store.columns[1:all_sales_by_store.shape[1]]]

In [10]:
depart_sales_output_wide_lawn=depart_sales_output_wide.copy()
depart_sales_output_wide_lawn.columns=["location_id"]+["LawnGarden_"+str(name) for name in depart_sales_output_wide_lawn.columns[1:all_sales_by_store.shape[1]]]

In [11]:
depart_sales_output_wide_lawn=pd.merge(all_sales_by_store,depart_sales_output_wide_lawn,on="location_id",how="outer")
depart_sales_output_wide_lawn=depart_sales_output_wide_lawn.fillna(0)

In [12]:
depart_sales_output_wide.to_excel(writer,"Lawn_and_Garden",index=False)

In [13]:
depart_sales_output_wide_lawn.to_excel(writer,"Lawn_and_Garden_and_All",index=False)

In [14]:
writer.save()